In [ ]:
import random
import json
from models.db_models import TemplateModel
from llm_util import LLMProvider


def generate_story_verse_analysis(llmProvider : LLMProvider, template: TemplateModel) -> str:

    storyVerseAnalysisPrompt = template.storyVerseAnalysisPromptTemplate

    storyExcerptExamples = template.storyExcerptsExamples
    randomExcerpts = random.sample(storyExcerptExamples, min(3, len(storyExcerptExamples)))
    print(randomExcerpts)


    excerpts_str = json.dumps(randomExcerpts)
    response_id = llmProvider.initiateResponse(storyVerseAnalysisPrompt)

    print(f"Submitted LLM request, response ID: {response_id}")
    response = llmProvider.getPooledResponse(response_id)
    print(f"Response: {response.output[0].content[0].text}")
    return response.output[0].content[0].text



In [ ]:
from pymongo import MongoClient
from models.db_models import TemplateModel, Job
import os
from dotenv import load_dotenv
from llm_util import OpenAiLLMProvider


if __name__ == "__main__":
    llmProvider = OpenAiLLMProvider()
    load_dotenv()
    mongo_uri = os.getenv('MONGODB_URI')
    client = MongoClient(mongo_uri)
    db_name = os.getenv('MONGODB_DATABASE') or 'sherlock-v2'
    db = client[db_name]
    llmProvider = OpenAiLLMProvider()
    template = db.prompt_templates.find_one({"storyVerse": os.getenv('STORY_VERSE')})
    if template is None:
        raise ValueError("No template found for the given storyVerse.")
    template = TemplateModel(**template)
    ## create new job entry
    job = Job(story_analysis="")
    
    # STEP 1 - storyverse analysis
    storyVerseAnalysis = generate_story_verse_analysis(llmProvider, template)
    job.story_analysis = storyVerseAnalysis
    job_id = db.jobs.insert_one(job.model_dump()).inserted_id
    print(f"Job saved with ID: {job_id}")

['Example1']
Submitted LLM request, response ID: resp_68bde90369bc819cbea516a305a9c7c806919db1cb3172a4
Response: Hello! Yes, I'm doing well, thank you for asking. How can I assist you today?
Job saved with ID: 68bde90b6d2b97872e34f473
